<center>
    <img src="https://fpt.edu.vn/Resources/brand/uploads/749540_132829686029858301_o.jpg" width="500" alt="cognitiveclass.ai logo"  />
</center>

# Lab 2: Access SQL Database Server using R




<br>

#### Class name: AI1706

#### Student code: HE170707

#### Student name: Phạm Thế Hưng

<br>


## Objectives

After completing this lab you will be able to:

*   Create connection with SQL Server using RODBC
*   Query data from the table
*   Retrieve the result set into a dataframe
*   Close the database connection

Estimated time needed: **90** minutes
<h4 style='color:red; font-weight:bold'>NOTE:</h4>
<h5 style='color:red; font-weight:bold'>
    <p>- Do not cheat! 1 point for anybody copy or share code</p>
    <p>- Rename your file with format: "class_mssv.ipynb" before submit(e.g AI1706_namnhHE123456)</p>
</h5>

**Notice:** Please follow the instructions given by lecture to import data from .csv file to SQL database server

## Install and import the `RODBC` library

<a href="https://cran.r-project.org/web/packages/RODBC/index.html">RODBC API</a> implements odbc database connectivity with compliant databases where drivers exist on the host system. 

Two groups of commands are provided. 

**odbc*** commands implement relatively low level access to the odbc functions of similar name. The odbc* group return -1 in stat on error. Up to 16 connections can be open at once to any combination of dsn/hosts. Columns are limited to 255 chars of non-binary data. The functions where usage is obvious from the name are not described below.

**sql*** commands are higher level constructs to read, save, copy and manipulate data between data frames and sql tables. In general sql* commands return a data frame on success, or -1/verbose on error depending on the errors parameter. 


**If the RODBC package is not installed in your system, please uncomment line below and run the cell to install the package:**

In [1]:
install.packages("RODBC")

Installing package into ‘/home/sofia/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



In [2]:
library(RODBC)

When the command above completes, the `pyodbc` and `pandas` library is loaded in your notebook.

## Identify the database connection properties

Connecting to SQL database server requires the following information:

*   Driver Name
*   Server
*   Database
*   User ID
*   User Password


In [4]:
#Replace the placeholder values with the actuals for your SQL Server connection properties
connection <- paste("Driver={PostgreSQL Unicode}",
                          "Server=localhost",
                          "Database=testdb",
                          "User=postgresql",
                          "password=12345678",
                          "trusted_connection=yes",
                          sep=";")
conn <- odbcDriverConnect(connection)
print("connected to server!")

[1] "connected to server!"


## Understand the datasets

To complete the assignment problems in this notebook you will be using three datasets that are available on the city of Chicago's Data Portal:

1.  <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01">Socioeconomic Indicators in Chicago</a>
2.  <a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01">Chicago Public Schools</a>
3.  <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01">Chicago Crime Data</a>

### 1. Socioeconomic Indicators in Chicago

This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
[https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2](https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
[https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

### 3. Chicago Crime Data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
[https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)


### Download the datasets

This assignment requires you to have these three tables populated with a subset of the whole datasets.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the links below to download and save the datasets (.CSV files):

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Chicago Census Data</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Chicago Public Schools</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Chicago Crime Data</a>

**NOTE**: For the learners who are encountering issues with loading from .csv in DB2 on Firefox, you can download the .txt files and load the data with those:

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.txt?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Chicago Census Data</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.txt?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Chicago Public Schools</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.txt?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Chicago Crime Data</a>

**NOTE:** Ensure you have downloaded the datasets using the links above instead of directly from the Chicago Data Portal. The versions linked here are subsets of the original datasets and have some of the column names modified to be more database friendly which will make it easier to complete this assignment.


In [29]:
census <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.txt?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01"
destfile_census <- "./census.csv"
public <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.txt?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01"
destfile_public <- "./public.csv"
crime <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.txt?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01"
destfile_crime <- "./crime.csv"

download.file(census, destfile_census, method = "auto")
download.file(public, destfile_public, method = "auto")
download.file(crime, destfile_crime, method = "auto")

# Check if the files have been downloaded successfully
file.exists(destfile_census)
file.exists(destfile_public)
file.exists(destfile_crime)


[1] TRUE

[1] TRUE

[1] TRUE

In [39]:
census <- read.csv("./census.csv")
crime <- read.csv("./crime.csv");
public <- read.csv("./public.csv");

In [28]:
create_census_table_query <- "
  CREATE TABLE census (
    COMMUNITY_AREA_NUMBER INT,
    COMMUNITY_AREA_NAME VARCHAR(255),
    PERCENT_OF_HOUSING_CROWDED FLOAT,
    PERCENT_HOUSEHOLDS_BELOW_POVERTY FLOAT,
    PERCENT_AGED_16__UNEMPLOYED FLOAT,
    PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA FLOAT,
    PERCENT_AGED_UNDER_18_OR_OVER_64 FLOAT,
    PER_CAPITA_INCOME INT,
    HARDSHIP_INDEX INT
  );"

# Execute the CREATE TABLE query
sqlQuery(conn, create_census_table_query)
sqlSave(conn, census, tablename = "census", rownames = FALSE, append = TRUE)

[1] "42P07 1 ERROR: relation \"census\" already exists;\nError while executing the query"                                                                                                                                                                                                                                                                                                                                                  
[2] "[RODBC] ERROR: Could not SQLExecDirect '\n  CREATE TABLE census (\n    COMMUNITY_AREA_NUMBER INT,\n    COMMUNITY_AREA_NAME VARCHAR(255),\n    PERCENT_OF_HOUSING_CROWDED FLOAT,\n    PERCENT_HOUSEHOLDS_BELOW_POVERTY FLOAT,\n    PERCENT_AGED_16__UNEMPLOYED FLOAT,\n    PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA FLOAT,\n    PERCENT_AGED_UNDER_18_OR_OVER_64 FLOAT,\n    PER_CAPITA_INCOME INT,\n    HARDSHIP_INDEX INT\n  );'"

In [42]:
sqlSave(conn, public, tablename = "public", rownames = FALSE, append = TRUE)
sqlSave(conn, census, tablename = "census", rownames = FALSE, append = TRUE)
sqlSave(conn, crime, tablename = "crime", rownames = FALSE, append = TRUE)

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.

In [46]:
ques1 <- sqlQuery(conn, "SELECT COUNT(*) FROM CRIME;"); ques1

,count
,<int>
1,533


### Problem 2

##### List community areas with per capita income less than 11000.



In [48]:
#Enter your code below
ques2 <- sqlQuery(conn, "SELECT COMMUNITY_AREA_NAME FROM CENSUS WHERE PER_CAPITA_INCOME < 11000"); ques2

,community_area_name
,<chr>
1,West Garfield Park
2,South Lawndale
3,Fuller Park
4,Riverdale
5,West Garfield Park
6,South Lawndale
7,Fuller Park
8,Riverdale
9,West Garfield Park


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)


In [49]:
#Enter your code below
ques3 <- sqlQuery(conn, "SELECT case_number FROM CRIME WHERE description ILIKE '%minor%'"); ques3

,case_number
,<chr>
1,HL266884
2,HK238408


### Problem 4

##### List all kidnapping crimes involving a child?


In [51]:
#Enter your code below
ques4 <- sqlQuery(conn, "SELECT * FROM CRIME WHERE primary_type = 'KIDNAPPING' AND description ILIKE '%child%'");ques4

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,⋯,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,year,latitude,longitude,location
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>
1,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,⋯,15,29,25,20,1143050,1897546,2007,41.87491,-87.75025,"(41.874908413, -87.750249307)"


### Problem 5

##### What kinds of crimes were recorded at schools?


In [53]:
#Enter your code below
ques5 <- sqlQuery(conn, "SELECT DISTINCT(primary_type) FROM CRIME WHERE location_description ILIKE '%school%'");ques5

,primary_type
,<chr>
1,ASSAULT
2,BATTERY
3,CRIMINAL DAMAGE
4,CRIMINAL TRESPASS
5,NARCOTICS
6,PUBLIC PEACE VIOLATION


### Problem 6

##### List the average safety score for each type of school.

In [77]:
#Enter your code below
ques6 <- sqlQuery(conn, 'SELECT "Elementary_Middle_or_High_School", AVG("SAFETY_SCORE") AS avg_safety_score FROM PUBLIC GROUP BY "Elementary_Middle_or_High_School"');ques6

[1] "42703 1 ERROR: column \"Elementary_Middle_or_High_School\" does not exist;\nError while executing the query"                                                                              
[2] "[RODBC] ERROR: Could not SQLExecDirect 'SELECT \"Elementary_Middle_or_High_School\", AVG(\"SAFETY_SCORE\") AS avg_safety_score FROM PUBLIC GROUP BY \"Elementary_Middle_or_High_School\"'"

In [71]:
sqlQuery(conn, 'SELECT "Elementary, Middle, or High School" FROM PUBLIC;')

[1] "42703 1 ERROR: column \"Elementary, Middle, or High School\" does not exist;\nError while executing the query"
[2] "[RODBC] ERROR: Could not SQLExecDirect 'SELECT \"Elementary, Middle, or High School\" FROM PUBLIC;'"

### Problem 7

##### List 5 community areas with highest % of households below poverty line


In [ ]:
#Enter your code below


### Problem 8

##### Which community area is most crime prone?


In [ ]:
#Enter your code below


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index

In [ ]:
#Enter your code below


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [ ]:
#Enter your code below


### Close database connection

In [ ]:
close(connection)

## Summary

In this lab you established a connection to a database instance of SQL Server Warehouse from a Jupyter notebook using `RODBC` API. Then query some data from database server. You also retrieved the data into a dataframe.


## Author

<a href="" target="_blank">Do Thai Giang</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2022-09-28        | 1.0     | Giangdt26  | Create the 1st version             |
| 2022-09-29        | 1.1     | Giangdt26  | Add student information section    |
| 2022-09-29        | 1.2     | Giangdt26  | Remove personal infor              |

<hr>

## <h3 align="center"> © FPT University. All rights reserved. <h3/>
